# **Detekcija linija**

U funkciji ispod vrši se detekcija linija. Posmatra se prvi frejm i pomoću Canny algoritma se detektuju ivice na slici. Potrebno je ovo uraditi samo na jednom frejmu od svakog videa jer linije uvek imaju isti položaj tokom videa. Ujedno, to predstavlja predprocesiranje za Hough transformaciju. Nakon toga se vrši Hough transformacija, koja nam daje četiri prave, tačnije tačke kojima su određene prave. Povratna vrednost su četiri prave, po dve za svaku liniju koja treba da se detektuje.

In [ ]:
def detekcijaLinija(video):
    cap = cv2.VideoCapture(video) #Ucitavanje videa
    success,image = cap.read()
    count = 0
    success = True
    ret = []
    while success:
      success, image = cap.read()
      if count == 0: #Prvi frejm
        edges = cv2.Canny(image,100,200) #Nalazenje ivica
        
        lines = cv2.HoughLinesP(edges,1,np.pi/180,100,minLineLength=199,maxLineGap=10) #Detektovanje linija
        for line in lines:
            x1, y1, x2, y2 = line[0]
            tacka = []
            tacka.append(x1)
            tacka.append(y1)
            tacka.append(x2)
            tacka.append(y2)
            ret.append(tacka);
            cv2.line(image, (x1, y1), (x2, y2), (0, 255, 255), 1)
        cv2.waitKey(0)
        
      count += 1
    cap.release()
    cv2.destroyAllWindows()
    return ret

In [ ]:
def dilate(image):
    kernel = np.ones((2,2)) # strukturni element 2x2 blok
    return cv2.dilate(image, kernel, iterations=1)
def erode(image):
    kernel = np.ones((3,3)) # strukturni element 3x3 blok
    return cv2.erode(image, kernel, iterations=1)

# **Detekcija brojeva**
U ovom odeljku je odrađena detekcija brojeva. U funckiji detekcijaBrojeva se detektuju svi brojevi u frejmu, dok se šum i linije zanemaruju. Broj se dalje šalje neuronskoj mreži, kako bi se odredilo koji je tačno broj u pitanju. Kako bi se poboljšali rezultati detektovanja cifara, svaka kontura(tj. svaki detektovan broj) je smešten u gornji levi ugao slike dimenzija 28x28. Isto je urađeno i sa slikama cifara iz MNIST-a.

In [ ]:
def detekcijaBrojeva(video, plavaLinijaVideo1, zelenaLinijaVideo1):
    cap = cv2.VideoCapture(video) #Ucitavanje videa
    #success,image = cap.read()
    count = 0
    resenje = 0
    #success = True
    ret = []
    retBr = []
    while True:
        success, image = cap.read()
        if not success: 
            break
      #if count == 0: #Prvi frejm
        gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY) #Siva slika
        ret,gray = cv2.threshold(gray,200,255,0) #Sve sto nije bela boja se brise, tj dobijaju se samo brojevi
        gray2 = gray.copy()
        samoBrojevi = Image.fromarray(gray2, 'L') #Samo brojevi na frejmu 
        #samoBrojevi.show() #Prikaz brojeva sa frejma 
        konture, hier = cv2.findContours(gray2,cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)
        for kontura in konture:
            if 10<cv2.contourArea(kontura)<200:
                (x,y,sirina,visina) = cv2.boundingRect(kontura)
                if(visina>12 and visina<50 and sirina>1): 
                    #cv2.rectangle(image,(x,y),(x+sirina,y+visina),(0, 255, 0),1, 1) #Uoviri detektovane brojeve
                    broj = np.array(samoBrojevi) 
                    broj = broj[y:y+visina,x:x+sirina]
                    mask = broj>0
                    isecen = broj[np.ix_(mask.any(1),mask.any(0))]
                    img = np.zeros((28,28))
                    for j in range(0,28):
                        for k in range(0,28):
                            if(j<len(isecen) and k<len(isecen[0])):
                                img[j][k] = isecen[j][k]
                    imgarr = img.reshape(1, 28*28)
                    predicted_classes = model.predict_classes(imgarr)
                    x = Broj(kontura, predicted_classes, count, x, y, sirina, visina, 0, 0, img)
                    ret = dodajeSeUListuMogucihPrelazaka(image, img, x, plavaLinijaVideo1, zelenaLinijaVideo1)
                    if ret is not None: 
                        resenje += ret
                        
        gray2 = dilate(dilate(gray2))
        cv2.imshow(video, image)
        #display_video(image)
        cv2.waitKey(1)
        count += 1
    cap.release()
    cv2.destroyAllWindows()
    return gray2, resenje

In [ ]:
import time 
def dodajeSeUListuMogucihPrelazaka(image, img, x, plavaLinijaVideo1, zelenaLinijaVideo1):
    uspeo = 0
    ret = 0
    if not moguciBrojevi:
        moguciBrojevi.append(x)
    for i, broj in enumerate(moguciBrojevi):
        try: 
            istiBroj = distance((broj.x, broj.y), (x.x, x.y))
            if(istiBroj<18): #12
                x.predikcija = broj.predikcija
                x.presaoP = broj.presaoP
                x.presaoZ = broj.presaoZ
                x.frame = broj.frame
                x.slika = broj.slika
                moguciBrojevi[i] = x
                uspeo = 1
        except: 
            uspeo = 0
    if(uspeo == 0):
            moguciBrojevi.append(x)
    for j, br in enumerate(moguciBrojevi):
        start = (plavaLinijaVideo1[0], plavaLinijaVideo1[1]) #Pocetak linije 
        end = (plavaLinijaVideo1[2], plavaLinijaVideo1[3]) #Kraj linije
        line_vec = vector(start, end) #Vektor linije
        tacka = (moguciBrojevi[j].x + br.width, br.y + br.height) 
        dist1 = 0
        dist1, nearest, r = pnt2line(tacka, start, end) 
        start = (zelenaLinijaVideo1[0], zelenaLinijaVideo1[1]) #Pocetak linije 
        end = (zelenaLinijaVideo1[2], zelenaLinijaVideo1[3]) #Kraj linije
        line_vec = vector(start, end) #Vektor linije
        tacka = (moguciBrojevi[j].x + br.width, br.y + br.height) 
        dist2 = 0
        dist2, nearest, r = pnt2line(tacka, start, end)
        if(dist1 < 4 and dist1 >= 0  and br.presaoP == 0):
            slikaBroja = Image.fromarray(br.slika)
            #slikaBroja.show()
            moguciBrojevi[j].presaoP = 1
            ret += int(moguciBrojevi[j].predikcija)

        if(dist2 < 4 and dist2 >= 0  and br.presaoZ == 0):
            slikaBroja = Image.fromarray(br.slika)
            #slikaBroja.show()
            #print(nearest)
            moguciBrojevi[j].presaoZ = 1
            ret -= int(moguciBrojevi[j].predikcija)
        
    return ret

In [ ]:
def pnt2line(pnt, start, end):
    line_vec = vector(start, end)
    pnt_vec = vector(start, pnt)
    line_len = length(line_vec)
    line_unitvec = unit(line_vec)
    pnt_vec_scaled = scale(pnt_vec, 1.0/line_len)
    t = dot(line_unitvec, pnt_vec_scaled)    
    if t < 0.0:
        t = 0.0
    elif t > 1.0:
        t = 1.0
    nearest = scale(line_vec, t)
    dist = distance(nearest, pnt_vec)
    nearest = add(nearest, start)
    return (dist, nearest)

In [ ]:
def treniranjeModela():
    #MNIST baza 
    (X_train, y_train), (X_test, y_test) = mnist.load_data()
    
    x = np.concatenate((X_train, X_test)) #Spajanje trening i test skupa, zarad vise podataka za obucavanje
    y = np.concatenate((y_train, y_test))

    X_train = x
    y_train = y

    for i in range(0, 70000):
        konture, hier = cv2.findContours(X_train[i],cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE) #Nalazenje broja na slici za obucavanje 
        try:
            kontura = konture[0]
            if (1<cv2.contourArea(kontura)<200):
                (x,y,sirina,visina) = cv2.boundingRect(kontura)
                mask = X_train[i]>0
                isecen = X_train[i][np.ix_(mask.any(1),mask.any(0))]
                img = np.zeros((28,28))
                for j in range(0,28):
                    for k in range(0,28):
                        if(j<len(isecen) and k<len(isecen[0])):
                            img[j][k] = isecen[j][k]
                        X_train[i, j, k] = img[j, k]
                slika = Image.fromarray(X_train[i])
        except:
            print("Doslo je do greske prilikom pomeranja broja u levi gornji ugao")

    slika = Image.fromarray(X_train[0])
    slika.show()
    
    # building the input vector from the 28x28 pixels
    X_train = X_train.reshape(70000, 784)
    X_test = X_test.reshape(10000, 784)
    X_train = X_train.astype('float32')
    X_test = X_test.astype('float32')

    X_train /= 255
    X_test /= 255

    # one-hot encoding using keras' numpy-related utilities
    n_classes = 10
    Y_train = np_utils.to_categorical(y_train, n_classes)
    Y_test = np_utils.to_categorical(y_test, n_classes)

    # building a linear stack of layers with the sequential model
    model.add(Dense(512, input_shape=(784,)))
    model.add(Activation('relu'))                            
    model.add(Dropout(0.2))

    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dropout(0.2))

    model.add(Dense(10))
    model.add(Activation('softmax'))


    # compiling the sequential model
    model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')
    # training the model and saving metrics in history
    history = model.fit(X_train, Y_train,
              batch_size=128, epochs=20,
              verbose=2,
              validation_data=(X_test, Y_test))
    return X_test, Y_test, y_test

In [ ]:
class resenjaVidea:
    def __init__(nazivVidea, resenje):
        self.nazivVidea = nazivVidea
        self.resenje= resenje

In [ ]:
class Broj:
    def __init__(self, kontura, predikcija, frame, x, y, width, height, presaoP, presaoZ, slika):
        self.kontura = kontura
        self.predikcija= predikcija
        self.frame = frame
        self.x = x
        self.y = y
        self.width = width
        self.height = height
        self.presaoP = presaoP
        self.presaoZ = presaoZ
        self.slika = slika
    def __repr__(self):
         return "<Test predikcija:%s frame:%s presao:%s> " % (self.predikcija, self.frame, self.presao)
    def __str__(self):
         return "<Test predikcija:%s frame:%s presao:%s x:%s y:%s width:%s height:%s> " % (self.predikcija, self.frame, self.presao, self.x, self.y, self.width, self.height)

In [ ]:
import math
def dot(v,w):
    x,y = v
    X,Y = w
    return x*X + y*Y
  
def length(v):
    x,y = v
    return math.sqrt(x*x + y*y)
  
def vector(b,e):
    x,y = b
    X,Y = e
    return (X-x, Y-y)
  
def unit(v):
    x,y = v
    mag = length(v)
    return (x/mag, y/mag)
  
def distance(p0,p1):
    return length(vector(p0,p1))

def scale(v,sc):
    x,y = v
    return (x * sc, y * sc)
  
def add(v,w):
    x,y = v
    X,Y = w
    return (x+X, y+Y)
  
def pnt2line(pnt, start, end):
    line_vec = vector(start, end)
    pnt_vec = vector(start, pnt)
    line_len = length(line_vec)
    line_unitvec = unit(line_vec)
    pnt_vec_scaled = scale(pnt_vec, 1.0/line_len)
    t = dot(line_unitvec, pnt_vec_scaled)    
    indikator = 1 #da li je broj zapravo iznad ili levo ili desno od linije |__|
    if t < 0.0:
        t = 0.0
        indikator = -1
    elif t > 1.03:
        t = 1.0
        indikator = -1
    nearest = scale(line_vec, t)
    dist = distance(nearest, pnt_vec)
    nearest = add(nearest, start)
    return (dist, (int(nearest[0]), int(nearest[1])), indikator)

In [ ]:
def procenti():
    res = []
    n = 0
    with open('res.txt') as file:	
        data = file.read()
        lines = data.split('\n')
        for id, line in enumerate(lines):
            if(id>0):
                cols = line.split('\t')
                if(cols[0] == ''):
                    continue
                cols[1] = cols[1].replace('\r', '')
                res.append(float(cols[1]))
                n += 1

    correct = 0
    student = []
    student_results = []
    with open("out.txt") as file:
        data = file.read()
        lines = data.split('\n')
        for id, line in enumerate(lines):
            cols = line.split('\t')
            if(cols[0] == ''):
                continue
            if(id==0):
                student = cols  
            elif(id>1):
                cols[1] = cols[1].replace('\r', '')
                student_results.append(float(cols[1]))

    diff = 0
    for index, res_col in enumerate(res):
        diff += abs(res_col - student_results[index])
    percentage = 100 - abs(diff/sum(res))*100

# **Main**
Ovaj segment predstavlja main programa. Prvo se vrši detekcija linija. Kao glavna prava se uzima ona koja je ispod, jer je potrebno da broj pređe preko nje kako bi bio sumiran.

In [ ]:
import cv2
import numpy as np
from numpy import array
import matplotlib
import sys, os
matplotlib.use('agg')
import matplotlib.pyplot as plt
from PIL import Image

from keras.datasets import mnist
from keras.models import Sequential, load_model
from keras.layers.core import Dense, Dropout, Activation
from keras.utils import np_utils

model = Sequential()
X_test, Y_test, y_test= treniranjeModela()

videi = ['video-0.avi', 'video-1.avi', 'video-2.avi', 'video-3.avi', 'video-4.avi', 'video-5.avi', 'video-6.avi', 'video-7.avi', 'video-8.avi', 'video-9.avi']
res = []
for i in videi:
    moguciBrojevi = []
    resenje = 0
    linijeVideo = detekcijaLinija(i)
    if(linijeVideo[0][0] < linijeVideo[2][0]):
        zelenaLinijaVideo = linijeVideo[0]
        plavaLinijaVideo = linijeVideo[2]
    else:
        zelenaLinijaVideo = linijeVideo[2]
        plavaLinijaVideo = linijeVideo[0]
    brojevi, resenje = detekcijaBrojeva(i, plavaLinijaVideo, zelenaLinijaVideo)
    res.append(resenje)
    
with open ('out.txt', 'w') as outfile:
    outfile.write('RA 81/2015 Jelena Vesovic\nfile sum\n')
    for i in range(0,len(res)):
        sumica = float(res[i])
        outfile.write(videi[i] +'\t'+ str(sumica) + '\n')
    
procenti()
    